In [1]:
import pixiedust

Pixiedust database opened successfully


In [2]:
from pixiedust.display.app import *

@PixieApp
class HelloWorldPixieApp:
    @route()
    def main(self):
        return"""
            <input pd_options="clicked=true" type="button" value="Click Me">
        """
    @route(clicked="true")
    def _clicked(self):
        return """
            <input pd_options="clicked=false" type="button" value="You Clicked, Now Go back">
        """
#run the app
HelloWorldPixieApp().run(runInDialog='false')

In [3]:
from pixiedust.display.app import *
import pandas as pd
import numpy as np

@PixieApp
class HelloWorldPixieAppWithData:
    @route()
    def main(self):
        return"""
        <div class="row">
           <div class="col-sm-2">
              <input pd_options="handlerId=dataframe"
                     pd_entity
                     pd_target="target{{prefix}}"
                     type="button" value="Preview Data">
           </div>
           <div class="col-sm-10" id="target{{prefix}}"/>
        </div>
        """

dates = pd.date_range('20130101',periods=6)
df = pd.DataFrame(np.random.randn(6,4),index=dates,columns=list('ABCD'))
    
#run the app
HelloWorldPixieAppWithData().run(df, runInDialog='false')

## Weather Forecast in a PixieApp

The code for the PixieApp is divided into a few cells to make it easier to interpret:

- CSS for the PixieApp
- JS for the PixieApp
- HTML for the PixieApp
- PixieApp Python module

In [4]:
import pandas as pd
df = df = pd.read_csv('data/df.csv')
current_df = pd.read_csv('data/current_df.csv')
forecast_df = pd.read_csv('data/forecast_df.csv')

In [5]:
# the CSS for the PixieApp
# - provides some style

weatherAppCSS = """
  .pixiedust-app.pixiedust-maximize .centerContainer {
    padding: 0 !important;
  }
  .pixiedust-app.pixiedust-maximize .expandButton {
    display: none;
  }
  .pixiedust-app .modal-header {
    background-color: rgba(0,0,0,0.45); /*#01021F;*/
    border-bottom: 1px solid #00b4a0;
    padding: 10px 0 10px 0;
  }
  .pixiedust-app .modal-title {
    color: #fff;
    font-size: 18px;
    padding-left: 20px;
  }
  .pixiedust .modal-header .close {
    color: #00b4a0;
    font-size: 30px;
    font-weight: 400;
    margin: -5px 60px 0 0;
  }
  .pixiedust-app .modal-content {
    background-image: url(https://images.unsplash.com/photo-1454702762838-a6df8cc3517f);
    background-repeat: no-repeat;
    background-size: cover;
    border-top: 0 none;
  }
  .pixiedust-app .modal-content::after {
    top: 0;
    left: 0;
    height: 100%;
    width: 100%;
    content: "";
    position: absolute;
    display: block;
    opacity: 0.5;
    background: -webkit-linear-gradient(left, #1d3649 0%, #325C80 50%);
    background: linear-gradient(to right, #1d3649 0%, #325C80 50%);
  }
  .pixiedust-app .modal-content > div {
    position: relative;
    z-index: 1;
  }
  .weather-app {
    border: none;
    height: 600px;
    margin: 0;
    padding: 0;
    position: relative;
  }
  .pixiedust-app.pixiedust-maximize .weather-app {
    height: 100%;
  }
  .weather-app > .row  {
    margin: 0;
    padding: 0;
    position: relative;
  }
  .weather-app > .row:first-child {
    height: 200px;
  }
  .weather-app > .row:last-child {
    height: calc(100% - 205px);
  }
  .weather-app > .row > div {
    height: 100%;
    margin: 0;
    padding: 10px;
    position: relative;
  }
  .weather-app .app-info,
  .weather-app .app-chart {
    background-color: rgba(0, 0, 0, 0.55);
    color: #fff;
    height: 100%;
    margin: 0;
    padding: 10px;
    position: relative;
  }
  .weather-app .app-title {
    font-size: 20px;
    color: #fff;
    font-weight: 200;
    border-bottom: 1px solid #00b4a0;
    padding-bottom: 5px;
  }
  .weather-app .app-title > div:first-child,
  .weather-app .title-target > div {
    display: none;
  }
  .weather-app .app-title > div:last-child {
    visibility: hidden;
  }
  .weather-app .app-copy {
    padding: 10px;
    font-size: 18px;
  }
  .weather-app #charts{{prefix}} {
    height: calc(100% - 45px);
    position: relative;
    width: 100%;
  }
  .weather-app #map{{prefix}} {
    height: calc(100% - 55px);
    margin-top: 30px;
    position: relative;
    width: 100%;
  }
  .weather-app .btn-group {
    border: 0 none;
    bottom: 20px;
    padding: 0;
    position: absolute;
    right: 20px;
  }
  .weather-app .btn {
    background-color: #ab236f !important;
    border: 2px solid #ab236f !important;
    color: #fff !important;
    font-weight: 400 !important;
    margin: 0 !important;
    min-width: 150px;
  }
  .weather-app .dropdown-menu {
    border: 0 none;
    border-radius: 0;
    width: 100%;
  }
  .weather-app .dropdown-menu li {
    color: #000 !important;
  }
  .weather-app .dropdown-menu li:hover {
    background-color: #ab236f;
    color: #fff !important;
    cursor: pointer;
  }
    """

In [6]:
# the HTML for the PixieApp
# - provides the HTML structure

weatherAppHTML = """
<div class="weather-app weather-app{{prefix}}" pixiedust="{{pd_controls|htmlAttribute}}">
  <div class="row">
    <div class="col-xs-12">
      <div class="app-info">
        <div class="app-title">Regional Weather Conditions</div>
        <div class="app-copy">
          <p>This is a PixieApp utilizing The Weather Company API. The timeline in the lower right pane shows a forecast
             for the selected weather condition and city. The Mapbox map in the lower left pane shows the selected weather
             condition for various predefined locations. Clicking on a location will update the timeline
             for the selected city.</p>
          <div class="btn-group">
            <button class="btn btn-default btn-lg dropdown-toggle" type="button" data-toggle="dropdown" aria-haspopup="true" aria-expanded="false">
              <span class="title-target" id="selected{{prefix}}" pd_script="print(self.currentChartInfo['label'])">Select a weather condition</span>
              <i class="fa fa-chevron-down" aria-hidden="true"></i>
            </button>
            <ul class="dropdown-menu">
              {% for info in this.measures %}
              <li data-info-value="{{info['valueField']}}" pd_script="self.currentChartInfo={{ info }}" onclick="javascript:updateSelected(this)">
                  {{info["label"]}}
                  <target pd_target="selected{{prefix}}" pd_refresh="selected{{prefix}}"/>
                  <target pd_target="chartstitle{{prefix}}" pd_refresh="chartstitle{{prefix}}"/>
                  <target pd_target="maptitle{{prefix}}" pd_refresh="maptitle{{prefix}}"/>
                  <target pd_target="charts{{prefix}}" pd_refresh="charts{{prefix}}"/>
                  <target pd_target="map{{prefix}}" pd_refresh="map{{prefix}}"/>
              </li>
              {% endfor %}
            </ul>
          </div>
        </div>
      </div>
    </div>
  </div>
  <div class="row" id="alltherows{{prefix}}">
    <div class="col-xs-6">
      <div class="app-chart">
        <div id="maptitle{{prefix}}" class="app-title" pd_script="print('{} across the Regions'.format(self.currentChartInfo['label']))">Recent Weather Condition</div>
        <div id="map{{prefix}}" pd_entity="pixieapp_entity[1]" pd_options="$val(mapOptions)"></div>
      </div>
    </div>
    <div class="col-xs-6">
      <div class="app-chart">
        <div id="chartstitle{{prefix}}" class="app-title" pd_script="print('{} Forecast for {}'.format(self.currentChartInfo['label'], self.selectedCity))">Weather Condition Forecast</div>
        <div id="charts{{prefix}}" pd_entity="chartDF" pd_options="$val(chartOptions)"></div>
      </div>
    </div>
    <pd_event_handler pd_origin="map{{prefix}}" pd_script="self.loadCityDF(eventInfo['place'])">
      <target pd_target="chartstitle{{prefix}}" pd_refresh="chartstitle{{prefix}}"/>
      <target pd_target="charts{{prefix}}" pd_refresh="charts{{prefix}}"/>
    </pd_event_handler>
  </div>
</div>
    """

In [7]:
# @hidden_cell
# Mapbox API access token: https://www.mapbox.com/studio/account/tokens/

mapBoxToken = 'pk.eyJ1IjoicmFqcnNpbmdoIiwiYSI6ImNqM2s4ZDg4djAwcGYyd3BwaGxwaDV3bWoifQ.d5Rklkdu5MeGAnXu1GMNYw'

In [8]:
# the JS for the PixieApp
# - provides the options for the mapbox map
# - provides the options for the timeline
# - updates the selected weather conditions dropdown

weatherAppJS = """
  function mapOptions() {{
    return "rendererId=mapbox;handlerId=mapView;keyFields=lat,lon;valueFields="
      + $('li[data-info-value].selected, weather-app{{{{prefix}}}}').data('infoValue')
      + ";kind=choropleth;basemap=light-v9;colorrampname=Light to Dark Red;coloropacity=100;preserveCols=place;sortby=place;legend=true"
      + ";mapboxtoken={0}"
  }}
  function chartOptions() {{
    return "rendererId=bokeh;handlerId=lineChart;keyFields=date;aggregation=AVG;valueFields="
      + $('li[data-info-value].selected, weather-app{{{{prefix}}}}').data('infoValue')
      + ";legend=false;rowCount=10000;noChartCache=true;no_margin=true;chartsize=100"
  }}
  function updateSelected(n) {{
    $(n).addClass('selected').siblings('.selected').removeClass('selected')
    $(n).closest(".dropdown-menu").prev().dropdown("toggle")
  }}
""".format(mapBoxToken)

In [9]:
from pixiedust.display.app import *

@PixieApp
class WeatherApp:
    def getDialogOptions(self):
        return {
            'title': '#ODSC London 2017',
            'maximize': 'true'
        }
    
    def setup(self):
#         self.url = 'https://github.com/ibm-cds-labs/python-notebooks/blob/master/weathericons/icon'+str(icon)+'.png?raw=true'
        self.measures = [
            {"label": "Temperature [F]", "valueField": "temperature"},
            {"label": "Temperature [C]", "valueField": "temperatureC"},
            {"label": "Chance of Rain [%]", "valueField": "chance of rain"},
            {"label": "Relative Humidity [%]", "valueField": "humidity"},
            {"label": "Wind Speed [mph]", "valueField": "wind speed"}
        ]
        self.loadCityDF('London')
        
    def loadCityDF(self, place):
        self.selectedCity = place
        self.cityDF = forecast_df.loc[forecast_df["place"] == place]
    @property
    def chartDF(self):
        return self.cityDF if hasattr(self, "cityDF") else forecast_df.loc[forecast_df["place"]=='London']
        
    @route()
    def main(self):
        return '<style>{}</style><script>{}</script>{}'.format(weatherAppCSS, weatherAppJS, weatherAppHTML)
    
# run the app
dfs = [forecast_df, current_df]

WeatherApp().run(dfs, runInDialog='true')    

In [10]:
%pixiedustLog -l debug

2017-11-21 09:24:43,748 - pixiedust.utils.storage - INFO - No change in version: 1.1.2 -> 1.1.2.
2017-11-21 09:24:50,500 - pixiedust.display - DEBUG - Creating a new display handler with options {'nostore_pixieapp': 'HelloWorldPixieApp_instance', 'nostore_ispix': 'true', 'runInDialog': 'false', 'handlerId': '__main___HelloWorldPixieApp_id'}: <pixiedust.display.display.CellHandshakeMeta object at 0x10fdadd30>
2017-11-21 09:24:51,256 - pixiedust.display - DEBUG - Creating a new display handler with options {'nostore_pixieapp': 'HelloWorldPixieApp_instance', 'nostore_ispix': 'true', 'runInDialog': 'false', 'handlerId': '__main___HelloWorldPixieApp_id', 'cell_id': 'EAA199EB45154CD7BD7A603B7A2FDDC9', 'nostore_pixiedust': 'true', 'nostore_cw': '1098', 'org_params': 'nostore_pixieapp,nostore_ispix,runInDialog,handlerId,nostore_pixiedust', 'nostore_bokeh': 'true', 'showchrome': 'true'}: <abc.__main___HelloWorldPixieApp_Meta object at 0x10fdadb70>
2017-11-21 09:24:51,345 - pixiedust.display - D